In [5]:
import os
import sys

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import spacy
import seaborn as sns

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler



In [2]:
nlp = spacy.load("en_core_web_lg")

In [10]:
data_train = pd.read_csv('cve_train.csv')
data_test = pd.read_csv('cve_test.csv')

description_train = data_train['description']
description_test = data_test['description']
cvss_train = data_train['base_score']
cvss_test = data_test['base_score']


In [12]:
print(description_train.head())

0    A remote code exection vulnerability was ident...
1    An information disclosure vulnerability in the...
2    The JPEGSetupEncode function in tiff_jpeg.c in...
3    Cloud Foundry UAA, versions prior to 74.0.0, i...
4    This affects the package ini before 1.3.6. If ...
Name: description, dtype: object


In [13]:
#transform tokens of a description and summarize them
vectors = description_train.apply(lambda row: sum([token.vector for token in nlp(row)]))

In [17]:
description_train_vectors = list(vectors)
reg = SVR(kernel='rbf')
reg.fit(description_train_vectors, cvss_train)

SVR()

In [18]:
#put test data into a list
list_description = []
for i in range(len(description_test)):
    list_description.append(description_test.iloc[i])

In [19]:
list_description_vec = []
for desc in list_description:
    list_description_vec.append(sum([token.vector for token in nlp(desc)]))

In [20]:
cvss_pred = []
for desc_vec in list_description_vec:
    cvss_pred.append(reg.predict([desc_vec]))

In [21]:
cvss_true = []
for i in range(len(cvss_test)):
    cvss_true.append(cvss_test.iloc[i])

In [35]:
#evaluation
from sklearn import metrics
print("Explained variance score (1 is the best): %.3f" %metrics.explained_variance_score(cvss_true, cvss_pred))
print("Max error: %.3f" %metrics.max_error(cvss_true, cvss_pred))
print("Mean absolute percentage error: %.3f" %metrics.mean_absolute_percentage_error(cvss_true, cvss_pred))
print("Correlation coefficient: %.3f" %metrics.r2_score(cvss_true, cvss_pred))



Explained variance score (1 is the best): 0.390
Max error: 6.115
Mean absolute percentage error: 0.154
Correlation coefficient: 0.389


In [33]:
for i in range(0, 20):
    print(cvss_pred[i])
    print(cvss_true[i])
    print()

[7.4661249]
7.5

[7.63278205]
7.8

[8.22439322]
8.8

[4.14713951]
4.3

[5.15562947]
5.5

[6.89972806]
7.0

[7.88468669]
4.7

[7.53229076]
9.8

[5.87283644]
5.5

[7.77672928]
9.1

[8.50735935]
9.8

[6.63063347]
7.2

[5.19119846]
4.7

[7.59984634]
7.8

[6.01000603]
4.8

[8.70402406]
8.8

[7.59104949]
7.8

[7.3477621]
8.4

[5.61356849]
8.7

[6.19843648]
7.3



In [ ]:
import pickle
pickle.dump(reg, open('svm_model', 'wb'))